# Setup the Agent Development Kit

<img src="img/agent_development_kit_logo.png" align="left" style="height: 100px;">Agent Development Kit (ADK) is a flexible and modular framework for developing and deploying AI agents. While optimized for Gemini and the Google ecosystem, ADK is model-agnostic, deployment-agnostic, and is built for compatibility with other frameworks. ADK was designed to make agent development feel more like software development, to make it easier for developers to create, deploy, and orchestrate agentic architectures that range from simple tasks to complex workflows.

## Basic Setup

### Define Notebook Variables

Update the variables below to match your environment. You will be prompted for the AlloyDB password you chose then you provisioned the environment with Terraform.

In [ ]:
# Project variables
project_id = "your-project"
region = "your-region"
vpc = "demo-vpc"
gcs_bucket_name = f"project-files-{project_id}"
cloud_sql_instance = "my-postgres-instance"

In [ ]:
# Set env variable to suppress annoying system warnings when running shell commands
%env GRPC_ENABLE_FORK_SUPPORT=1

### Connect to your Google Cloud Project

In [ ]:
# Configure gcloud.
!gcloud config set project {project_id}

### Configure Logging

In [ ]:
import logging
import sys

# Configure the root logger to output messages with ERROR level
logging.basicConfig(level=logging.ERROR, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Install Dependencies

In [ ]:
! pip install --quiet toolbox-core==0.2.1 \
                      google-adk==1.5.0 \
                      google-genai==1.23.0


## Build an Agent with MCP Toolbox Tools

### Get the Toolbox Endpoint

In [ ]:
import json

toolbox_url = ! gcloud run services describe toolbox --region {region} --format 'value(metadata.annotations."run.googleapis.com/urls")'
toolbox_url = json.loads(toolbox_url[0])[0]
print(toolbox_url)

### Get Auth Token

We required authenticated invocations of the Cloud Run service, so we first need to grab an auth token to use with our ToolboxClient. 

In [ ]:
import urllib

import google.auth.transport.requests
import google.oauth2.id_token


def get_auth_token(endpoint):
    # Cloud Run uses your service's hostname as the `audience` value
    # audience = 'https://my-cloud-run-service.run.app/'
    # For Cloud Run, `endpoint` is the URL (hostname + path) receiving the request
    # endpoint = 'https://my-cloud-run-service.run.app/my/awesome/url'
    
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, endpoint)

    return id_token

### Define Agent

In [ ]:
# Reference: https://googleapis.github.io/genai-toolbox/getting-started/local_quickstart/

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google import genai
from google.genai import types
from toolbox_core import ToolboxSyncClient

import asyncio
import os

# Use GOOGLE_API_KEY or Vertex AI
# os.environ['GOOGLE_API_KEY'] = 'your-api-key'

os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'TRUE'
os.environ['GOOGLE_CLOUD_PROJECT']      = project_id
os.environ['GOOGLE_CLOUD_LOCATION']     = region

# Get auth token to invoke Toolbox on Cloud Run
auth_token = get_auth_token(toolbox_url)

async def invoke_agent(query: str):
  with ToolboxSyncClient(
        toolbox_url,
        client_headers={"Authorization": f"Bearer {auth_token}"}
  ) as toolbox_client:

      prompt = """
        You're a helpful financial assistant. You handle fraud detection tasks, transaction lookups,
        account details, loan payments, suspicious activity reports, and other tasks related to 
        financial data. Always provide a customer_id when using tools to lookup information.
      """

      app_name = 'finance_agent'
    
      user_id = '123'

      root_agent = Agent(
          model='gemini-2.5-flash',
          name=app_name,
          description='A helpful AI assistant.',
          instruction=prompt,
          tools=toolbox_client.load_toolset("finance-toolset"),
      )

      session_service = InMemorySessionService()
      artifacts_service = InMemoryArtifactService()
      session = await session_service.create_session(
          state={}, app_name=app_name, user_id=user_id
      )
      runner = Runner(
          app_name=app_name,
          agent=root_agent,
          artifact_service=artifacts_service,
          session_service=session_service,
      )

      content = types.Content(role='user', parts=[types.Part(text=query)])
      events = runner.run(session_id=session.id,
                          user_id=user_id, new_message=content)

      responses = (
        part.text
        for event in events
        for part in event.content.parts
        if part.text is not None
      )

      for text in responses:
        print(text)


### Manually Invoke the Agent

In [ ]:
prompts = [
    "What are the latest trasactions for customer_id 11?",
    "Tell me about the recent account transfers made by customer_id 8 and flag any suspicious activity.",
    "Is anything strange about the latest tranactions made by customer_id 3?",
]

for p in prompts:
    print(f"User prompt: {p}\n")
    print("Agent response:")
    response = await invoke_agent(p)
    print("\n\n")
    

## Inspect Agent Processing

The examples above show that the Agent has access to our databases via MCP Toolbox, because the output contains data the Agent wouldn't otherwise know. But what if we want to debug an Agent's logic or understand the specific tool calls that were made? When developing locally, we can set the log level to info to see detailed, step-by-step Agent execution and tool call details. 

### Set log level to INFO

In [ ]:
import logging
import sys

# Configure the root logger to output messages with INFO level or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Examine Detailed Output

In [ ]:
prompt = "How many transfers has customer_id 462 made recently? Is there anything strange about the transfers?"
await invoke_agent(prompt)

## Deploy the ADK Agent to Cloud Run

Now we will deploy the ADK agent to Cloud Run. The deployment process will use a `Dockerfile` to containerize the application and configure it to connect to the Cloud SQL instance for session management. We will also enable Cloud Logging and Cloud Trace for observability.

References:
- https://github.com/alphinside/deploy-and-manage-adk-service/tree/main/weather_agent
- https://codelabs.developers.google.com/deploy-manage-observe-adk-cloud-run#1

### Persist Session State in Cloud SQL

The Agent Development Kit (ADK) uses a Session Service to maintain the state of conversations. By default, it uses an in-memory session service, but this is not suitable for production environments where the server may restart, and sessions would be lost. To ensure persistence, we can connect the ADK to a Cloud SQL database to act as the persistent store for the Session service.

In [ ]:
# Get Cloud SQL connection name
cloud_sql_connection_name = !gcloud sql instances describe {cloud_sql_instance} --format="value(connectionName)"
cloud_sql_connection_name = cloud_sql_connection_name[0]

# Construct the SESSION_SERVICE_URI
session_service_uri = f"postgresql+pg8000://postgres:YOUR_PASSWORD@/postgres?unix_sock=/cloudsql/{cloud_sql_connection_name}/.s.PGSQL.5432"
print(f"SESSION_SERVICE_URI: {session_service_uri}")


### Create `__init__.py`

In [ ]:
# Initialize the agent directory
! mkdir -p deploy
! mkdir -p deploy/finance_agent

In [ ]:
%%writefile deploy/finance_agent/__init__.py
# __init__.py

from finance_agent.agent import root_agent

__all__ = ["root_agent"]

### Create `agent.py`

In [ ]:
%%writefile deploy/finance_agent/agent.py
import os
import google.auth
import google.auth.transport.requests
import google.oauth2.id_token
from google.adk.agents import Agent
from toolbox_core import ToolboxSyncClient

# --- Environment and Authentication ---

# It's best practice to set these in the Cloud Run service's environment variables
# instead of hardcoding them in the script.
# os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'TRUE'
# os.environ['GOOGLE_CLOUD_PROJECT']      = 'your-project-id'
# os.environ['GOOGLE_CLOUD_LOCATION']     = 'your-region'

def get_auth_token(endpoint: str) -> str:
    """Fetches an OIDC token for authenticating with a Cloud Run service."""
    auth_req = google.auth.transport.requests.Request()
    return google.oauth2.id_token.fetch_id_token(auth_req, endpoint)

# The URL for the toolbox service should be passed as an environment variable
# for better flexibility between different environments (dev, prod).
TOOLBOX_URL = os.getenv("TOOLBOX_URL")
if not TOOLBOX_URL:
    raise ValueError("TOOLBOX_URL environment variable not set.")

# Get an auth token to invoke the Toolbox on Cloud Run
auth_token = get_auth_token(TOOLBOX_URL)

# --- Agent Definition ---

# Initialize the Toolbox client once and reuse it
toolbox_client = ToolboxSyncClient(
    TOOLBOX_URL,
    client_headers={"Authorization": f"Bearer {auth_token}"}
)

# Define the agent's instructions
prompt = """
You're a helpful financial assistant. You handle fraud detection tasks, transaction lookups,
account details, loan payments, suspicious activity reports, and other tasks related to
financial data. Always provide a customer_id when using tools to lookup information.
"""

# Define the root agent
root_agent = Agent(
    model='gemini-2.5-flash',
    name='finance_agent',
    description='A helpful AI assistant for financial tasks.',
    instruction=prompt,
    # Load the tools from your MCP Toolbox
    tools=toolbox_client.load_toolset("finance-toolset"),
)

### Create `server.py`

In [ ]:
%%writefile deploy/server.py
import os
from fastapi import FastAPI
from google.adk.cli.fast_api import get_fast_api_app
from google.cloud import logging as google_cloud_logging
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider, export
from tracing import CloudTraceLoggingSpanExporter # Assumes you have a tracing.py file in the root directory

# --- Server Setup ---

# Initialize the Google Cloud Logging client
logging_client = google_cloud_logging.Client()
logger = logging_client.logger(__name__)

# The directory where your agent.py and __init__.py are located
AGENT_DIR = os.path.dirname(os.path.abspath(__file__))

# Get the session service URI from environment variables.
# This is where you'll provide the connection string for Cloud SQL.
session_uri = os.getenv("SESSION_SERVICE_URI")

# Prepare arguments for the FastAPI app
app_args = {"agents_dir": AGENT_DIR, "web": True}

# Use the Cloud SQL session service if the URI is provided
if session_uri:
    app_args["session_service_uri"] = session_uri
    logger.log_text(f"Using database session service.", severity="INFO")
else:
    logger.log_text(
        "SESSION_SERVICE_URI not provided. Using in-memory session service. "
        "Sessions will be lost on server restart.",
        severity="WARNING",
    )

# --- Observability: Tracing ---
# Enable Cloud Trace

provider = TracerProvider()
# The CloudTraceLoggingSpanExporter is a custom exporter defined in tracing.py
# to handle trace data limits. Make sure you have the 'tracing.py' file.
processor = export.BatchSpanProcessor(CloudTraceLoggingSpanExporter())
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)


# --- Create and Configure the FastAPI App ---

# Create the FastAPI app using the ADK utility
app: FastAPI = get_fast_api_app(**app_args)

app.title = "finance-agent"
app.description = "API for interacting with the ADK Finance Agent"

# The ADK's get_fast_api_app automatically creates the necessary
# endpoints like /run_sse for you to interact with the agent.
# You can add custom endpoints here if needed.

### Create `tracing.py`

In [ ]:
%%writefile deploy/tracing.py
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import json
import logging
from collections.abc import Sequence
from typing import Any

import google.cloud.storage as storage
from google.cloud import logging as google_cloud_logging
from opentelemetry.exporter.cloud_trace import CloudTraceSpanExporter
from opentelemetry.sdk.trace import ReadableSpan
from opentelemetry.sdk.trace.export import SpanExportResult


class CloudTraceLoggingSpanExporter(CloudTraceSpanExporter):
    """
    An extended version of CloudTraceSpanExporter that logs span data to Google Cloud Logging
    and handles large attribute values by storing them in Google Cloud Storage.

    This class helps bypass the 256 character limit of Cloud Trace for attribute values
    by leveraging Cloud Logging (which has a 256KB limit) and Cloud Storage for larger payloads.
    """

    def __init__(
        self,
        logging_client: google_cloud_logging.Client | None = None,
        storage_client: storage.Client | None = None,
        bucket_name: str | None = None,
        debug: bool = False,
        **kwargs: Any,
    ) -> None:
        """
        Initialize the exporter with Google Cloud clients and configuration.

        :param logging_client: Google Cloud Logging client
        :param storage_client: Google Cloud Storage client
        :param bucket_name: Name of the GCS bucket to store large payloads
        :param debug: Enable debug mode for additional logging
        :param kwargs: Additional arguments to pass to the parent class
        """
        super().__init__(**kwargs)
        self.debug = debug
        self.logging_client = logging_client or google_cloud_logging.Client(
            project=self.project_id
        )
        self.logger = self.logging_client.logger(__name__)
        self.storage_client = storage_client or storage.Client(project=self.project_id)
        self.bucket_name = bucket_name or f"{self.project_id}-weather-agent-logs-data"
        self.bucket = self.storage_client.bucket(self.bucket_name)

    def export(self, spans: Sequence[ReadableSpan]) -> SpanExportResult:
        """
        Export the spans to Google Cloud Logging and Cloud Trace.

        :param spans: A sequence of spans to export
        :return: The result of the export operation
        """
        for span in spans:
            span_context = span.get_span_context()
            trace_id = format(span_context.trace_id, "x")
            span_id = format(span_context.span_id, "x")
            span_dict = json.loads(span.to_json())

            span_dict["trace"] = f"projects/{self.project_id}/traces/{trace_id}"
            span_dict["span_id"] = span_id

            span_dict = self._process_large_attributes(
                span_dict=span_dict, span_id=span_id
            )

            if self.debug:
                print(span_dict)

            # Log the span data to Google Cloud Logging
            self.logger.log_struct(
                span_dict,
                labels={
                    "type": "agent_telemetry",
                    "service_name": "weather-agent",
                },
                severity="INFO",
            )
        # Export spans to Google Cloud Trace using the parent class method
        return super().export(spans)

    def store_in_gcs(self, content: str, span_id: str) -> str:
        """
        Initiate storing large content in Google Cloud Storage/

        :param content: The content to store
        :param span_id: The ID of the span
        :return: The  GCS URI of the stored content
        """
        if not self.storage_client.bucket(self.bucket_name).exists():
            logging.warning(
                f"Bucket {self.bucket_name} not found. "
                "Unable to store span attributes in GCS."
            )
            return "GCS bucket not found"

        blob_name = f"spans/{span_id}.json"
        blob = self.bucket.blob(blob_name)

        blob.upload_from_string(content, "application/json")
        return f"gs://{self.bucket_name}/{blob_name}"

    def _process_large_attributes(self, span_dict: dict, span_id: str) -> dict:
        """
        Process large attribute values by storing them in GCS if they exceed the size
        limit of Google Cloud Logging.

        :param span_dict: The span data dictionary
        :param trace_id: The trace ID
        :param span_id: The span ID
        :return: The updated span dictionary
        """
        attributes = span_dict["attributes"]
        if len(json.dumps(attributes).encode()) > 255 * 1024:  # 250 KB
            # Separate large payload from other attributes
            attributes_payload = dict(attributes.items())
            attributes_retain = dict(attributes.items())

            # Store large payload in GCS
            gcs_uri = self.store_in_gcs(json.dumps(attributes_payload), span_id)
            attributes_retain["uri_payload"] = gcs_uri
            attributes_retain["url_payload"] = (
                f"https://storage.mtls.cloud.google.com/"
                f"{self.bucket_name}/spans/{span_id}.json"
            )

            span_dict["attributes"] = attributes_retain
            logging.info(
                "Length of payload span above 250 KB, storing attributes in GCS "
                "to avoid large log entry errors"
            )

        return span_dict

### Create `pyproject.toml`

In [ ]:
%%writefile deploy/pyproject.toml
[project]
name = "deploy-and-manage-adk-service"
version = "0.1.0"
description = "Add your description here"
readme = "README.md"
requires-python = ">=3.12"
dependencies = [
    "google-adk==1.3.0",
    "locust==2.37.10",
    "pg8000==1.31.2",
    "python-dotenv==1.1.0",
]

[dependency-groups]
dev = [
    "pytest==8.4.0",
    "ruff==0.11.13",
]

### Create a Dockerfile

In [ ]:
%%writefile deploy/Dockerfile
FROM python:3.12-slim
RUN pip install --no-cache-dir uv==0.7.13
WORKDIR /app
COPY . .
RUN uv sync
EXPOSE 8080
CMD ["uv", "run", "uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8080"]


### Deploy the Agent to Cloud Run

In [ ]:
!gcloud run deploy finance-agent --quiet --no-user-output-enabled \
    --source ./deploy/ \
    --network={vpc} \
    --subnet={vpc} \
    --ingress=internal \
    --port 8080 \
    --project {project_id} \
    --region {region} \
    --no-allow-unauthenticated \
    --add-cloudsql-instances={cloud_sql_connection_name} \
    --update-env-vars=TOOLBOX_URL={toolbox_url},GOOGLE_GENAI_USE_VERTEXAI=True,SESSION_SERVICE_URI={session_service_uri}

### Invoke the Cloud Run Agent

In [ ]:
import requests
import json
import uuid
from google.colab import auth
import google.auth.transport.requests
import google.oauth2.id_token

# --- Get Service URL and Auth Token ---

# Get the URL of your deployed Cloud Run service
service_url = !gcloud run services describe finance-agent --region {region} --format 'value(status.url)'
service_url = service_url[0]

# Get an OIDC token to authenticate with the Cloud Run service
auth.authenticate_user()
auth_req = google.auth.transport.requests.Request()
id_token = google.oauth2.id_token.fetch_id_token(auth_req, service_url)

# --- Prepare and Send the Request ---

invoke_url = f"{service_url}/run_sse"
headers = {
    "Authorization": f"Bearer {id_token}",
    "Content-Type": "application/json",
    "Accept": "text/event-stream"
}

# Each conversation needs a unique session ID
session_id = str(uuid.uuid4())

# The data payload for the agent
data = {
    "new_message": {
        "content": {
            "parts": [{"text": "What are the latest transactions for customer_id 11?"}]
        }
    },
    "session_id": session_id,
    "user_id": "test-user-123"
}

# Send the request and process the streaming response
with requests.post(invoke_url, headers=headers, json=data, stream=True) as response:
    if response.status_code == 200:
        print("Agent Response:")
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                # SSE messages are prefixed with 'data: '
                if decoded_line.startswith('data: '):
                    try:
                        # Extract the JSON part of the message
                        json_data = json.loads(decoded_line[6:])
                        if 'content' in json_data and 'parts' in json_data['content']:
                            text = json_data['content']['parts'][0].get('text', '')
                            print(text, end='', flush=True)
                    except json.JSONDecodeError:
                        pass # Ignore lines that aren't valid JSON
        print("\n")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)